In [52]:
# !pip install yfinance
# !pip install tabulate

In [53]:
from datetime import datetime
from tabulate import tabulate
import pandas as pd

In [54]:
prices = pd.read_excel('/content/drive/MyDrive/AAPL_STOCKS_2023.xlsx')


prices.set_index('Date', inplace=True)
prices.drop('Field', axis = 1,  inplace=True)

prices

,FIBBG000B9XRY4
Date,
2023-01-03,124.2163
2023-01-04,125.4975
2023-01-05,124.1666
2023-01-06,128.7352
2023-01-09,129.2616
...,...
2023-12-22,193.3533
2023-12-26,192.8040
2023-12-27,192.9038


In [55]:
closing_prices = []

# Iterate over the series
for date, value in prices.iloc[:,0].items():
    closing_prices.append((str(date.date()), value))

print(closing_prices, len(closing_prices))



[('2023-01-03', 124.2163), ('2023-01-04', 125.4975), ('2023-01-05', 124.1666), ('2023-01-06', 128.7352), ('2023-01-09', 129.2616), ('2023-01-10', 129.8377), ('2023-01-11', 132.5788), ('2023-01-12', 132.4994), ('2023-01-13', 133.8402), ('2023-01-17', 135.0121), ('2023-01-18', 134.2871), ('2023-01-19', 134.3467), ('2023-01-20', 136.9289), ('2023-01-23', 140.1468), ('2023-01-24', 141.5571), ('2023-01-25', 140.8917), ('2023-01-26', 142.9774), ('2023-01-27', 144.9339), ('2023-01-30', 142.0239), ('2023-01-31', 143.3051), ('2023-02-01', 144.4373), ('2023-02-02', 149.7905), ('2023-02-03', 153.4454), ('2023-02-06', 150.6943), ('2023-02-07', 153.5944), ('2023-02-08', 150.883), ('2023-02-09', 149.8402), ('2023-02-10', 150.2082), ('2023-02-13', 153.0332), ('2023-02-14', 152.3866), ('2023-02-15', 154.5053), ('2023-02-16', 152.8939), ('2023-02-17', 151.7401), ('2023-02-21', 147.6917), ('2023-02-22', 148.1194), ('2023-02-23', 148.6068), ('2023-02-24', 145.9311), ('2023-02-27', 147.1346), ('2023-02-28

In [56]:
def find_returns_and_states(data):
    returns_and_states = []

    for i in range(1, len(data)):
        # Calculate daily returns
        returns = (data[i][1] - data[i-1][1]) / data[i-1][1]

        # Classify states
        if returns >= 0.01:
            state = 1
        elif returns > -0.01:
            state = 0
        else:
            state = -1

        # Append timestamp and state to the list
        returns_and_states.append((data[i][0], state))

    return returns_and_states

# Call the function with the list of tuples
states = find_returns_and_states(closing_prices)

len(states)

249

In [57]:
def calculate_transition_distribution(states):
    # Dictionary to store transition counts
    transition_counts = {
        1: {1: 0, 0: 0, -1: 0},
        0: {1: 0, 0: 0, -1: 0},
        -1: {1: 0, 0: 0, -1: 0}
    }

    for i in range(len(states) - 1):
        current_state = states[i][1]
        next_state = states[i+1][1]

        # Update transition counts
        transition_counts[current_state][next_state] += 1

    # Calculate transition probabilities
    transition_distribution = {}
    for state, transitions in transition_counts.items():
        total_transitions = sum(transitions.values())
        if total_transitions > 0:
            transition_probs = {next_state: count / total_transitions for next_state, count in transitions.items()}
        else:
            transition_probs = {next_state: 0 for next_state in transitions}
        transition_distribution[state] = transition_probs

    return transition_distribution

# Call the function with the 'states' list
transition_distribution = calculate_transition_distribution(states)

print(transition_distribution)

# Map integer keys to corresponding labels
label_map = {1: "Bullish", 0: "Flat", -1: "Bearish"}

# Convert the dictionary to a list of lists
table = []
for row_key in transition_distribution.keys():
    row = [label_map[row_key]]
    for col_key in transition_distribution[row_key].keys():
        row.append("{:.6f}".format(transition_distribution[row_key][col_key]))
    table.append(row)

# Print the labeled 2D list with grids
print("\nTransition Distribution: \n")
print(tabulate(table, headers=[""] + list(label_map.values()), tablefmt="grid"))



{1: {1: 0.19642857142857142, 0: 0.6785714285714286, -1: 0.125}, 0: {1: 0.25477707006369427, 0: 0.5987261146496815, -1: 0.1464968152866242}, -1: {1: 0.11428571428571428, 0: 0.7428571428571429, -1: 0.14285714285714285}}

Transition Distribution: 

+---------+-----------+----------+-----------+
|         |   Bullish |     Flat |   Bearish |
+=========+===========+==========+===========+
| Bullish |  0.196429 | 0.678571 |  0.125    |
+---------+-----------+----------+-----------+
| Flat    |  0.254777 | 0.598726 |  0.146497 |
+---------+-----------+----------+-----------+
| Bearish |  0.114286 | 0.742857 |  0.142857 |
+---------+-----------+----------+-----------+


In [58]:
# Calculating the optimal days to buy the stock inorder to maximise the portfolio value

def calculate_portfolio_value(states):
    V = 0  # Initial portfolio value
    buy_indices = []
    buy_dates = []
    state = "Flat"  # Initialize the current state as "Flat"

    # Initialize dictionary (memo) to hold the state combinations of current_day and next_day
    memo = {(1, 0): 1, (-1, 0): -1}

    # Iterate through each state transition
    for i in range(len(states) - 1):
        current_state = states[i][1]
        next_state = states[i + 1][1]

        # Check if the current state transition is already memoized
        if (next_state, current_state) in memo:
            V += memo[(next_state, current_state)]
            if memo[(next_state, current_state)] == 1:
                buy_indices.append(i + 1)  # Index of the day to buy
                buy_dates.append(states[i + 1][0]) # dates to buy

    return V, buy_indices, buy_dates


portfolio_value, optimal_buy_indices, optimal_buy_dates = calculate_portfolio_value(states)

print("\nOptimal portfolio value (V(N)): ", portfolio_value)
print("\nOptimal buy indices: ", optimal_buy_indices)
# print(len(optimal_buy_indices), len(optimal_buy_dates))
print("\nOptimal buy dates: ", optimal_buy_dates)



Optimal portfolio value (V(N)):  17

Optimal buy indices:  [5, 7, 11, 15, 20, 27, 29, 40, 49, 51, 58, 60, 68, 78, 84, 87, 93, 99, 102, 107, 109, 112, 116, 119, 122, 132, 141, 159, 163, 176, 186, 190, 206, 208, 211, 215, 217, 231, 233, 237]

Optimal buy dates:  ['2023-01-11', '2023-01-13', '2023-01-20', '2023-01-26', '2023-02-02', '2023-02-13', '2023-02-15', '2023-03-03', '2023-03-16', '2023-03-20', '2023-03-29', '2023-03-31', '2023-04-13', '2023-04-27', '2023-05-05', '2023-05-10', '2023-05-18', '2023-05-26', '2023-06-01', '2023-06-08', '2023-06-12', '2023-06-15', '2023-06-22', '2023-06-27', '2023-06-30', '2023-07-17', '2023-07-28', '2023-08-23', '2023-08-29', '2023-09-18', '2023-10-02', '2023-10-06', '2023-10-30', '2023-11-01', '2023-11-06', '2023-11-10', '2023-11-14', '2023-12-05', '2023-12-07', '2023-12-13']
